In [66]:
# Импортируем необходимые библиотеки
import pandas as pd
from IPython.display import display
from IPython.core.display import HTML

# Открываем файл Excel
file_path = 'dirty_data.xlsx'
data = pd.read_excel(file_path)

In [67]:
phrases = ['Вопрос решен', 'Вопрос не решен', 'Нравится качество выполнения заявки', 'Нравится качество работы сотрудников', 'Нравится скорость отработки заявок', 'Понравилось выполнение заявки']

# Создаем бинарные колонки для каждой фразы
for phrase in phrases:
    data[phrase] = data['taxonomy'].apply(lambda x: 1 if pd.notna(x) and phrase in x else 0)


# Удаляем ненужные колонки
delete_columns = ['annotation_id', 'annotator', 'id', 'created_at', 'taxonomy', 'lead_time', 'updated_at']
data.drop(columns=delete_columns, inplace=True)

# Объединяем колонки "Нравится качество выполнения заявки" и "Понравилось выполнение заявки"
# data['Нравится качество выполнения заявки'] = data['Нравится качество выполнения заявки'] | data['Понравилось выполнение заявки']

# Проверяем результат
print(f'Названия колонок: {", ".join(data.columns)}')
display(HTML(data.to_html(notebook=True, max_rows=10, max_cols=8)))
print(data.shape)

# Сохраняем результат в новый файл
data.to_excel('cleaned_data.xlsx', index=False)

Названия колонок: comment, rating, Вопрос решен, Нравится качество выполнения заявки, Нравится качество работы сотрудников, Нравится скорость отработки заявок, Понравилось выполнение заявки, Вопрос не решен


,comment,rating,Вопрос решен,Нравится качество выполнения заявки,Нравится качество работы сотрудников,Нравится скорость отработки заявок,Понравилось выполнение заявки,Вопрос не решен
0,спасибо,5,1,0,0,0,0,0
1,спасибо!,5,1,0,0,0,0,0
2,Отлично,5,1,0,0,0,0,0
3,Благодарю за оперативное решение проблемы !,5,1,0,0,1,0,0
4,Прекрасный специалист! Побольше таких,5,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...
2367,"долго, но приятно",5,1,0,0,0,0,0
2368,16.03.2024 г ребята установили кондиционер. ра...,5,1,1,1,1,0,0
2369,два дня мусоропровод. сегодня с утра тоже,5,0,0,0,0,0,1
2370,отрегулировать дверь замки . для должного откр...,5,0,0,0,0,0,1


(2372, 8)
